# Métodos de Ensemble


## Instabilidade

Vamos montar um modelo clássico para identificação de mensagens de spam. O modelo consiste em contar quantas vezes uma determinada palavra aparece na mensagem e classificá-la como spam ou não. **O foco da aula não é o modelo, mas estudar a estabilidade dele!**

### Coleta e limpeza de dados

In [ ]:
import pandas as pd
import string
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# dataset de mensagens de email
df = pd.read_csv('data/spam_ham.csv')

df.head()


In [ ]:
# um pouco de preprocessamento

def clean_text(df_input, column_to_clean):
    
    # converte para letras minusculas
    df_input['text_clean'] = df_input[column_to_clean].str.lower()
    
    # remove pontução
    remover = f"[{string.punctuation}]"
    df_input['text_clean'] = df_input['text_clean'].str.replace(remover, ' ', regex=True)
    
    # remove numeros
    numeros = f"[1234567890]"
    df_input['text_clean'] = df_input['text_clean'].str.replace(numeros, '', regex=True)
    
    return df_input['text_clean']

# criar coluna text clean
df['text_clean'] = clean_text(df, 'text')

df.head()

### Divisão do dataset

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.model_selection import train_test_split

def create_train_test_sets(df_input, semente):
    X = df_input['text_clean'].copy()
    y = df_input['type'].map({'ham':0, 'spam': 1})


    X_treino_bruto, X_teste_bruto, y_treino_, y_teste_ = train_test_split(X, y, 
                                                                          test_size=0.3, 
                                                                          random_state=semente)
    # preparação para a aula de NLP - exercicio do WordCount
    bag_of_words = CountVectorizer(max_features=100)
    bag_of_words.fit(X_treino_bruto)
    
    X_treino_ = bag_of_words.transform(X_treino_bruto)
    X_teste_ = bag_of_words.transform(X_teste_bruto)
    
    return X_treino_, X_teste_, y_treino_, y_teste_

# cria os datasets de treino e teste na prática
X_treino, X_teste, y_treino, y_teste = create_train_test_sets(df, 13)

In [ ]:
y_treino

In [ ]:
y_teste

In [ ]:
X_treino.todense()

In [ ]:
df.loc[1830, 'text_clean']

### Função para avaliar o modelo

In [ ]:
# encapsuland tudo na função!


In [ ]:
evaluate_model(df, tree_model, roc_auc_score, semente=13)

### O que acontece quando mudamos os dados de treino?

**Para pensar:** Só mudamos quais emails estão usando para treinar e a performance está instável!

### Enfim, as instabilidade!

In [ ]:
# função que sumariza os passos acima

def plot_scores(quantidade, modelo_eval, score_eval):
    sementes = [i for i in range(quantidade)]
    scores = [evaluate_model(df, modelo=modelo_eval, scoring_function=score_eval, semente=j) for j in sementes]
    
    sns.lineplot(x=sementes, y=scores)
    print(f"média: {np.mean(scores)}, desvio padrão: {np.std(scores)}")

In [ ]:
plot_scores(quantidade=15, modelo_eval=tree_model, score_eval=roc_auc_score)

# Bagging
Um meio de evitar a instabilidade é treinar diversos modelos **em paralelo** com **amostras** dos dados (técnia de **bootstrapping**) e combinar a decisão de todos eles no final. No caso de *regressão* fazemos a **média** dos resultados e para *classificação*, uma **votação**.

* Bootstrapping: Amostragem dos dados com reposição
* Modelos em paralelo: um modelo é idependente do outro
* Weak learner: Conjunto de amostra de dados + instancia do modelo


<img src="images/bagging_sketch.png">

In [ ]:
# implementação do BaggingClassifier no sklearn


In [ ]:
# será que é mais estável?


**Para pensar:** Qual o custo da estabilidade trazida pelo bagging?

>

In [ ]:
%%timeit
# verificar o tempo de forma ordenada


## Bagging of trees: **Random Forest**

Aplica a técnica de bootstrapping para criar diversas árvores!

- cada árvore é um weak learned! 

<img src="images/random_forest.png">

In [ ]:
# implementação do random forest no sklearn


A implementação do random forest é muito mais eficiente do que implementar um bagging de árvores na mão!

In [ ]:
%%timeit


# Boosting

Ao contrário do bagging, o método de boosting treina diversos modelos **em sequência**. 

<img src="images/boosting.png">




Essa técnica usa todos os dados e a cada iteração atribui diferentes pesos (importâncias) para os pontos que são classificados erroneamente (pode se tornar obcecado por outilers!). 

<img src="images/boosting1.jpeg">

<img src="https://i.stack.imgur.com/mQ9Np.png"/>

## Entendo os pesos!

In [ ]:
err_m = np.sort(np.random.random(100))

def a_m(x):
    return np.log((1-x)/x)

plt.plot(err_m, a_m(err_m))
plt.xlabel('err_m')
plt.ylabel('a_m')

## Algoritimos de boosting

### [AdaBoost](https://en.wikipedia.org/wiki/AdaBoost): Adaptative boosting

### [LightGBM](https://lightgbm.readthedocs.io/en/latest/Installation-Guide.html): Light Gradient Boosting Machine

pip install lightgbm

### [Xgboost](https://xgboost.readthedocs.io/en/latest/): Extreme gradient boosting

pip install xgboost

# Bonus: Stacking models - Para casa!

Para expandir nosso horizonte, vamos estender as idéais acima para a combinação de diferentes modelos, técnica conhecida como [Stacking](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingClassifier.html).